In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import  ColumnTransformer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
dataset =pd.read_csv('https://raw.githubusercontent.com/gogzicole/stage-f-07-heart-failure/master/data/heart_failure_clinical_records_dataset.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [5]:
dataset.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [26]:
X = dataset.drop(columns = 'DEATH_EVENT')
y = dataset['DEATH_EVENT']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )
y_train.value_counts()
x_test.count()

0    157
1     82
Name: DEATH_EVENT, dtype: int64

age                         60
anaemia                     60
creatinine_phosphokinase    60
diabetes                    60
ejection_fraction           60
high_blood_pressure         60
platelets                   60
serum_creatinine            60
serum_sodium                60
sex                         60
smoking                     60
time                        60
dtype: int64

In [8]:
import imblearn
from imblearn.over_sampling import SMOTE

In [28]:
smote = SMOTE(random_state=1)
X_balanced, y_balanced = smote.fit_sample(x_train, y_train)
X_balanced = pd.DataFrame(X_balanced, columns=x_train.columns)

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# standardscaler of the x_train
X_balanced_train = scaler.fit_transform(X_balanced)
X_balanced_train = pd.DataFrame(X_balanced_train, columns=x_train.columns)

In [30]:
# standardscaler of the x_test
x_tests = scaler.transform(x_test)
x_tests = pd.DataFrame(x_tests, columns=x_test.columns)

In [12]:
x_tests

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,-0.992356,1.229634,-0.489829,-0.765414,-1.435982,-0.653661,-0.718346,-0.641207,0.634970,0.824102,-0.565091,0.396124
1,-1.406737,-0.813250,-0.036955,1.306483,1.587283,-0.653661,3.219593,-0.472769,-0.978514,-1.213442,-0.565091,1.747545
2,-0.743728,1.229634,-0.513103,-0.765414,-1.435982,1.529845,1.829078,-0.135893,0.634970,-1.213442,-0.565091,-0.942304
3,-0.826604,-0.813250,-0.473343,-0.765414,-0.572192,-0.653661,-0.395747,-0.725426,-0.056523,0.824102,1.769627,-0.045687
4,0.250787,1.229634,-0.470434,-0.765414,-0.140297,1.529845,0.405190,-0.641207,-0.517519,0.824102,-0.565091,1.019857
5,0.250787,-0.813250,-0.439402,-0.765414,-0.572192,-0.653661,0.060342,-0.641207,0.404472,-1.213442,-0.565091,0.915901
6,-0.163594,1.229634,-0.352124,1.306483,-0.572192,-0.653661,-1.152187,-0.472769,0.173974,0.824102,1.769627,1.682573
7,-0.577975,-0.813250,-0.495647,-0.765414,-0.140297,-0.653661,0.004722,-0.388550,0.634970,0.824102,1.769627,-0.721398
8,0.665167,-0.813250,-0.483040,1.306483,0.723493,1.529845,0.338445,-0.220112,-0.056523,0.824102,1.769627,-1.163209
9,-1.406737,-0.813250,-0.302667,1.306483,2.019178,1.529845,1.372989,-0.472769,-0.056523,0.824102,-0.565091,0.915901


In [13]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None] 
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}


In [15]:

model = LGBMClassifier(random_state=1)
clf = GridSearchCV(model, hyperparameter_grid)
search = clf.fit(X_balanced_train, y_balanced)
#checking for the best parameter for the model
print(search.best_params_.values())
print(search.best_params_)

dict_values(['auto', 1, 2, 300])
{'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [16]:
param = LGBMClassifier(n_estimators=300, min_samples_split=2, 
                                 min_samples_leaf=1, max_features='auto', random_state = 1 )
param.fit(X_balanced_train, y_balanced)
y_pred = param.predict(x_tests)

LGBMClassifier(max_features='auto', min_samples_leaf=1, min_samples_split=2,
               n_estimators=300, random_state=1)

In [17]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report # Classification Report
print (confusion_matrix(y_test, y_pred))


print (classification_report(y_test,y_pred, digits=4))

# Accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print( 'Accuracy: {}' .format(accuracy )) 

[[39  7]
 [ 2 12]]
              precision    recall  f1-score   support

           0     0.9512    0.8478    0.8966        46
           1     0.6316    0.8571    0.7273        14

    accuracy                         0.8500        60
   macro avg     0.7914    0.8525    0.8119        60
weighted avg     0.8766    0.8500    0.8571        60

Accuracy: 0.85


In [18]:
X_balanced, y_balanced = smote.fit_sample(X, y)
X_balanced = pd.DataFrame(X_balanced, columns=x_train.columns)

In [19]:
make_column_transformer = [2,4,6,7,8,11]

pipe = Pipeline([('col_transform', ColumnTransformer(remainder = 'passthrough',
                transformers = [('scaler',MinMaxScaler(),make_column_transformer)])),
                ('grid_search',GridSearchCV(LGBMClassifier(),
                hyperparameter_grid, cv = 3, n_jobs = 2, scoring = 'accuracy', verbose = 1))])

new = pipe.named_steps['col_transform'].fit_transform(X_balanced) 

x_train, x_test,y_train, y_test = train_test_split(new, y_balanced, test_size = 0.2, random_state = 0)

pipe.fit(x_train,y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  54 tasks      | elapsed:   10.4s
[Parallel(n_jobs=2)]: Done 354 tasks      | elapsed:   32.1s
[Parallel(n_jobs=2)]: Done 854 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 1500 out of 1500 | elapsed:  1.8min finished


Pipeline(steps=[('col_transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  [2, 4, 6, 7, 8, 11])])),
                ('grid_search',
                 GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=2,
                              param_grid={'max_features': ['auto', 'sqrt',
                                                           'log2', None],
                                          'min_samples_leaf': [1, 2, 4, 6, 8],
                                          'min_samples_split': [2, 3, 5, 7, 9],
                                          'n_estimators': [50, 100, 300, 500,
                                                           1000]},
                              scoring='accuracy', verbose=1))])

In [20]:
model2 = pipe.named_steps['grid_search'].best_estimator_
model2.fit(x_train, y_train)
y_pred2 = model2.predict(x_test)

LGBMClassifier(max_features='auto', min_samples_leaf=1, min_samples_split=2,
               n_estimators=300)

In [21]:
print (confusion_matrix(y_test, y_pred2))


print (classification_report(y_test,y_pred2, digits=4))

# Accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred2)
print( 'Accuracy: {}' .format(accuracy )) 

[[30  5]
 [ 2 45]]
              precision    recall  f1-score   support

           0     0.9375    0.8571    0.8955        35
           1     0.9000    0.9574    0.9278        47

    accuracy                         0.9146        82
   macro avg     0.9187    0.9073    0.9117        82
weighted avg     0.9160    0.9146    0.9140        82

Accuracy: 0.9146341463414634


## StratifiedKFold

In [31]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

skf = StratifiedKFold(n_splits= 10 , shuffle= True , random_state= 1 )
f1_scores = []
#run for every split
for train_index, test_index in skf.split(X_balanced_train, y_balanced):
    x_train, x_test = np.array(X_balanced_train)[train_index], np.array(X_balanced_train)[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
model = LGBMClassifier().fit(x_train, y_train)
#save result to list
f1_scores.append(f1_score(y_true=y_test, y_pred= model.predict(x_test) ))

In [32]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report # Classification Report
print (confusion_matrix(y_true=y_test, y_pred= model.predict(x_test)))


print (classification_report(y_true=y_test, y_pred= model.predict(x_test), digits=4))

# Accuracy
accuracy = accuracy_score(y_true=y_test, y_pred= model.predict(x_test))
print( 'Accuracy: {}' .format(accuracy )) 

[[11  4]
 [ 0 16]]
              precision    recall  f1-score   support

           0     1.0000    0.7333    0.8462        15
           1     0.8000    1.0000    0.8889        16

    accuracy                         0.8710        31
   macro avg     0.9000    0.8667    0.8675        31
weighted avg     0.8968    0.8710    0.8682        31

Accuracy: 0.8709677419354839
